In [1]:
#setup + imports

import sys
sys.path.append('..')

import pandas as pd
import os
import re
from config import engine

# Helper function to extract uid from filename
def extract_uid(filename):
    """Extract uid from filenames like 'activity_u01.csv' -> 'u01'"""
    match = re.search(r'u\d+', filename)
    return match.group() if match else None

print("✓ Setup complete")

✓ Setup complete


In [9]:
# Load students from all sources (grades + sensing files)

def load_all_students():
    """Extract ALL unique uids from every data source"""
    all_uids = set()
    
    # From sensing files (extract from filenames)
    sensing_dirs = ['activity', 'audio', 'bluetooth', 'conversation', 'dark', 
                    'gps', 'phonecharge', 'phonelock', 'wifi', 'wifi_location']
    
    for subdir in sensing_dirs:
        dir_path = f'/Users/test/Downloads/Data Management & Databases/studentlife_project/data/sensing/{subdir}'
        if os.path.exists(dir_path):
            for filename in os.listdir(dir_path):
                uid = extract_uid(filename)
                if uid:
                    all_uids.add(uid)
    
    # From behavioral files
    for folder in ['sms', 'call_log', 'app_usage', 'calendar', 'dinning']:
        dir_path = f'../data/{folder}'
        if os.path.exists(dir_path):
            for filename in os.listdir(dir_path):
                uid = extract_uid(filename)
                if uid:
                    all_uids.add(uid)
    
    print(f"Found {len(all_uids)} total unique students across all files")
    print(f"UIDs found: {sorted(all_uids)}")
    
    # Check what's already in database
    existing = pd.read_sql("SELECT uid FROM student", engine)
    existing_uids = set(existing['uid'].tolist())
    
    # Find new ones to add
    new_uids = all_uids - existing_uids
    
    if new_uids:
        print(f"\nAdding {len(new_uids)} new students: {sorted(new_uids)}")
        new_students = pd.DataFrame({'uid': sorted(new_uids)})
        new_students.to_sql('student', engine, if_exists='append', index=False)
        print(f"✓ Successfully added {len(new_uids)} new students")
    else:
        print(f"✓ All {len(all_uids)} students already in database")
    
    # Verify final count
    final = pd.read_sql("SELECT COUNT(*) as count FROM student", engine)
    print(f"\nTotal students now in database: {final['count'][0]}")
    
    return len(all_uids)

load_all_students()

Found 49 total unique students across all files
UIDs found: ['u00', 'u01', 'u02', 'u03', 'u04', 'u05', 'u07', 'u08', 'u09', 'u10', 'u12', 'u13', 'u14', 'u15', 'u16', 'u17', 'u18', 'u19', 'u20', 'u22', 'u23', 'u24', 'u25', 'u27', 'u30', 'u31', 'u32', 'u33', 'u34', 'u35', 'u36', 'u39', 'u41', 'u42', 'u43', 'u44', 'u45', 'u46', 'u47', 'u49', 'u50', 'u51', 'u52', 'u53', 'u54', 'u56', 'u57', 'u58', 'u59']

Adding 19 new students: ['u00', 'u03', 'u13', 'u20', 'u23', 'u31', 'u34', 'u35', 'u36', 'u39', 'u42', 'u44', 'u45', 'u47', 'u50', 'u51', 'u53', 'u56', 'u58']
✓ Successfully added 19 new students

Total students now in database: 49


49

In [10]:
# Activity data

def load_activity_data():
    """Load all activity reading files"""
    activity_dir = '/Users/test/Downloads/Data Management & Databases/studentlife_project/data/sensing/activity'
    all_frames = []
    
    for filename in os.listdir(activity_dir):
        if not filename.endswith('.csv'):
            continue
        
        uid = extract_uid(filename)
        if not uid:
            print(f"⚠ Skipping {filename} - no uid found")
            continue
        
        filepath = os.path.join(activity_dir, filename)
        df = pd.read_csv(filepath)
        df['uid'] = uid
        
        # Fix column names (remove leading spaces)
        df.columns = df.columns.str.strip()
        
        # Rename to match schema
        df = df.rename(columns={'activity inference': 'activity_inference'})
        
        all_frames.append(df)
    
    # Combine all students
    combined = pd.concat(all_frames, ignore_index=True)
    
    # Load to database
    combined.to_sql('activity_reading', engine, if_exists='append', index=False)
    
    print(f"✓ Loaded {len(combined):,} activity readings from {len(all_frames)} students")
    return len(combined)

load_activity_data()

✓ Loaded 22,842,191 activity readings from 49 students


22842191

In [ ]:
# Audio data

def load_audio_data():
    """Load all audio reading files"""
    audio_dir = '/Users/test/Downloads/Data Management & Databases/studentlife_project/data/sensing/audio'
    all_frames = []
    
    for filename in os.listdir(audio_dir):
        if not filename.endswith('.csv'):
            continue
        
        uid = extract_uid(filename)
        if not uid:
            continue
        
        filepath = os.path.join(audio_dir, filename)
        df = pd.read_csv(filepath)
        df['uid'] = uid
        
        # Fix column names
        df.columns = df.columns.str.strip()
        df = df.rename(columns={'audio inference': 'audio_inference'})
        
        all_frames.append(df)
    
    combined = pd.concat(all_frames, ignore_index=True)
    combined.to_sql('audio_reading', engine, if_exists='append', index=False)
    
    print(f"✓ Loaded {len(combined):,} audio readings from {len(all_frames)} students")
    return len(combined)

load_audio_data()

In [ ]:
# Bluetooth data

def load_bluetooth_data():
    """Load all bluetooth scan files"""
    bt_dir = '/Users/test/Downloads/Data Management & Databases/studentlife_project/data/sensing/bluetooth'
    all_frames = []
    
    for filename in os.listdir(bt_dir):
        if not filename.endswith('.csv'):
            continue
        
        uid = extract_uid(filename)
        if not uid:
            continue
        
        filepath = os.path.join(bt_dir, filename)
        df = pd.read_csv(filepath)
        df['uid'] = uid
        
        # Rename to match schema (MAC -> mac, etc.)
        df = df.rename(columns={'MAC': 'mac'})
        
        all_frames.append(df)
    
    combined = pd.concat(all_frames, ignore_index=True)
    combined.to_sql('bluetooth_scan', engine, if_exists='append', index=False)
    
    print(f"✓ Loaded {len(combined):,} bluetooth scans from {len(all_frames)} students")
    return len(combined)

load_bluetooth_data()

In [ ]:
# Conversation data

def load_conversation_data():
    """Load all conversation files"""
    conv_dir = '/Users/test/Downloads/Data Management & Databases/studentlife_project/data/sensing/conversation'
    all_frames = []
    
    for filename in os.listdir(conv_dir):
        if not filename.endswith('.csv'):
            continue
        
        uid = extract_uid(filename)
        if not uid:
            continue
        
        filepath = os.path.join(conv_dir, filename)
        df = pd.read_csv(filepath)
        df['uid'] = uid
        
        # Fix column names
        df.columns = df.columns.str.strip()
        df = df.rename(columns={
            'start_timestamp': 'start_timestamp',
            ' end_timestamp': 'end_timestamp'
        })
        
        all_frames.append(df)
    
    combined = pd.concat(all_frames, ignore_index=True)
    combined.to_sql('conversation', engine, if_exists='append', index=False)
    
    print(f"✓ Loaded {len(combined):,} conversations from {len(all_frames)} students")
    return len(combined)

load_conversation_data()

In [ ]:
# GPS data

def load_gps_data():
    """Load all GPS files"""
    gps_dir = '/Users/test/Downloads/Data Management & Databases/studentlife_project/data/sensing/gps'
    all_frames = []
    
    for filename in os.listdir(gps_dir):
        if not filename.endswith('.csv'):
            continue
        
        uid = extract_uid(filename)
        if not uid:
            continue
        
        filepath = os.path.join(gps_dir, filename)
        df = pd.read_csv(filepath)
        df['uid'] = uid
        
        all_frames.append(df)
    
    combined = pd.concat(all_frames, ignore_index=True)
    combined.to_sql('gps_reading', engine, if_exists='append', index=False)
    
    print(f"✓ Loaded {len(combined):,} GPS readings from {len(all_frames)} students")
    return len(combined)

load_gps_data()

In [ ]:
# Dark period data

def load_dark_data():
    """Load all dark period files"""
    dark_dir = '/Users/test/Downloads/Data Management & Databases/studentlife_project/data/sensing/dark'
    all_frames = []
    
    for filename in os.listdir(dark_dir):
        if not filename.endswith('.csv'):
            continue
        
        uid = extract_uid(filename)
        if not uid:
            continue
        
        filepath = os.path.join(dark_dir, filename)
        df = pd.read_csv(filepath)
        df['uid'] = uid
        
        all_frames.append(df)
    
    combined = pd.concat(all_frames, ignore_index=True)
    combined.to_sql('dark_period', engine, if_exists='append', index=False)
    
    print(f"✓ Loaded {len(combined):,} dark periods from {len(all_frames)} students")
    return len(combined)

load_dark_data()

In [ ]:
# Phone charge data

def load_phonecharge_data():
    """Load all phone charge files"""
    charge_dir = '/Users/test/Downloads/Data Management & Databases/studentlife_project/data/sensing/phonecharge'
    all_frames = []
    
    for filename in os.listdir(charge_dir):
        if not filename.endswith('.csv'):
            continue
        
        uid = extract_uid(filename)
        if not uid:
            continue
        
        filepath = os.path.join(charge_dir, filename)
        df = pd.read_csv(filepath)
        df['uid'] = uid
        
        all_frames.append(df)
    
    combined = pd.concat(all_frames, ignore_index=True)
    combined.to_sql('phone_charge', engine, if_exists='append', index=False)
    
    print(f"✓ Loaded {len(combined):,} phone charge periods from {len(all_frames)} students")
    return len(combined)

load_phonecharge_data()

In [ ]:
# Phone lock data

def load_phonelock_data():
    """Load all phone lock files"""
    lock_dir = '/Users/test/Downloads/Data Management & Databases/studentlife_project/data/sensing/phonelock'
    all_frames = []
    
    for filename in os.listdir(lock_dir):
        if not filename.endswith('.csv'):
            continue
        
        uid = extract_uid(filename)
        if not uid:
            continue
        
        filepath = os.path.join(lock_dir, filename)
        df = pd.read_csv(filepath)
        df['uid'] = uid
        
        all_frames.append(df)
    
    combined = pd.concat(all_frames, ignore_index=True)
    combined.to_sql('phone_lock', engine, if_exists='append', index=False)
    
    print(f"✓ Loaded {len(combined):,} phone lock periods from {len(all_frames)} students")
    return len(combined)

load_phonelock_data()

In [ ]:
# Wifi data

def load_wifi_data():
    """Load all WiFi scan files"""
    wifi_dir = '/Users/test/Downloads/Data Management & Databases/studentlife_project/data/sensing/wifi'
    all_frames = []
    
    for filename in os.listdir(wifi_dir):
        if not filename.endswith('.csv'):
            continue
        
        uid = extract_uid(filename)
        if not uid:
            continue
        
        filepath = os.path.join(wifi_dir, filename)
        df = pd.read_csv(filepath)
        df['uid'] = uid
        
        # Rename to match schema (BSSID -> bssid)
        df = df.rename(columns={'BSSID': 'bssid'})
        
        all_frames.append(df)
    
    combined = pd.concat(all_frames, ignore_index=True)
    combined.to_sql('wifi_scan', engine, if_exists='append', index=False)
    
    print(f"✓ Loaded {len(combined):,} WiFi scans from {len(all_frames)} students")
    return len(combined)

load_wifi_data()

In [ ]:
# Wifi location data

def load_wifi_location_data():
    """Load all WiFi location files"""
    wifiloc_dir = '/Users/test/Downloads/Data Management & Databases/studentlife_project/data/sensing/wifi_location'
    all_frames = []
    
    for filename in os.listdir(wifiloc_dir):
        if not filename.endswith('.csv'):
            continue
        
        uid = extract_uid(filename)
        if not uid:
            continue
        
        filepath = os.path.join(wifiloc_dir, filename)
        df = pd.read_csv(filepath)
        df['uid'] = uid
        
        all_frames.append(df)
    
    combined = pd.concat(all_frames, ignore_index=True)
    combined.to_sql('wifi_location', engine, if_exists='append', index=False)
    
    print(f"✓ Loaded {len(combined):,} WiFi locations from {len(all_frames)} students")
    return len(combined)

load_wifi_location_data()

In [ ]:
# Summary

print("\n" + "="*50)
print("SENSING DATA LOAD COMPLETE")
print("="*50)
print("\nGo to DBeaver and check your tables!")
print("Try: SELECT COUNT(*) FROM activity_reading;")
